In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress
from pprint import pprint
from config import am_key
from census import Census
from us import states

# Census API Key
from config import c_key


c = Census(c_key, year=2018)

ModuleNotFoundError: No module named 'census'

In [ ]:
#Read in 4 parts of bus data csv

bus1 = pd.read_csv(r'../CSVs/filter_bus_1000.csv')
bus2 = pd.read_csv(r'../CSVs/new_bus_2.csv')
bus3 = pd.read_csv(r'../CSVs/filter_bus_3001-4500.csv')
bus4 = pd.read_csv(r'../CSVs/filter_bus_4500-5535.csv')

In [ ]:
#concat 4 csvs into 1 df with zipcodes and all bus stops
all_bus_stops = pd.concat([bus1, bus2, bus3, bus4])

In [ ]:
#delete unnecessary columns
del all_bus_stops['Zipcode']
del all_bus_stops['X']
del all_bus_stops['Y']
del all_bus_stops['NoName']
del all_bus_stops['NextRide']
del all_bus_stops['Juris']
del all_bus_stops['WaterFountain']
del all_bus_stops['ADA']
del all_bus_stops['ATIS_ID']
del all_bus_stops['Lat']
del all_bus_stops['Long']
del all_bus_stops['Routes']
del all_bus_stops['Zip']
del all_bus_stops['City']
del all_bus_stops['ComputedZipPlus4']
del all_bus_stops['Column1']
del all_bus_stops['StreetAddress']

del all_bus_stops['AUTO_UNIQUE_ID_2020-07-28_smavila1_new_bus_2']
del all_bus_stops['OBJECTID']
del all_bus_stops['DateUpdated']
del all_bus_stops['AUTO_UNIQUE_ID_2020-07-27_cceni03_filter_bus3001']
del all_bus_stops['AUTO_UNIQUE_ID_2020-07-28_czc5230_bus_4']
del all_bus_stops['AUTO_UNIQUE_ID_2020-07-27_pwewiora08_filter_bus_1000']
del all_bus_stops['Direction']
del all_bus_stops['State']
del all_bus_stops['Address']
del all_bus_stops['Type_']

del all_bus_stops['UpdatedReverseGeocoding']

del all_bus_stops['TimeTaken']
del all_bus_stops['TransactionId']
del all_bus_stops['Source']
del all_bus_stops['ErrorMessage']
del all_bus_stops['Version']
del all_bus_stops['ComputedAPN']


In [ ]:
#clean up & save new csv
all_bus_stops = all_bus_stops.rename(columns ={'ComputedZip':'Zipcode'})

all_bus_stops.to_csv(r'../PW_CSVs/all_bus_stops.csv')

all_bus_stops

In [ ]:
#read in CSV with ridership data
bus_riders = pd.read_csv(r'../CSVs/Valley_Metro_Bus_Ridership.csv')

#merge with bus stop df that has zipcodes
bus_riders = pd.merge(bus_riders, all_bus_stops, how = 'left', on = 'StopID')

In [ ]:
#delete unnecessary columns
del bus_riders['Location_x']

del bus_riders['Photo']
del bus_riders['Routes']
del bus_riders['OBJECTID']


del bus_riders['NextRide']
del bus_riders['Location_y']



In [ ]:
#save csv
bus_riders.to_csv(r'../PW_CSVs/bus_riders.csv')

bus_riders

In [ ]:
#run census API
census_data = c.acs5.get(('B19301_001E', 
                          'B01003_001E',"B17001_002E"), {'for': 'zip code tabulation area:*'})

census_pd = pd.DataFrame(census_data)

#B19301_001E = Per Capita Income
#B01003_001E = total population
#B17001_002E = poverty count

In [ ]:
# Column Renaming
census_pd = census_pd.rename(columns={"B19301_001E" : "Per Capita Income",
                                      "B17001_002E" : "Poverty Count",
                                      "B01003_001E" : "Population",
                                      "zip code tabulation area": "Zipcode"})
#remove NAN
census_pd = census_pd.fillna(0)

# Add in Poverty Rate (Poverty Count / Population)
census_pd["Poverty Rate"] = 100 * \
    census_pd["Poverty Count"].astype(
        int) / census_pd["Population"].astype(int)

#convert data types to be able to merge these two dfs 
census_pd['Zipcode'] = census_pd['Zipcode'].transform(float)

bus_riders['Zipcode']= pd.to_numeric(bus_riders['Zipcode'], errors='coerce').fillna(0).astype(float)


In [ ]:
#merge census data with bus ridership and bus stop data.
bus_census = pd.merge(bus_riders, census_pd, how='left', on=['Zipcode'])
bus_census.to_csv(r'../PW_CSVs/bus_census.csv')
bus_census.rename(columns = {'Day_Category': 'Day'}, inplace = True)

#cleaning up dataset
del bus_census['Q2016_2']
del bus_census['Q2016_3']
del bus_census['Q2016_4']
del bus_census['Q2017_1']
del bus_census['Q2017_2']
del bus_census['Q2017_3']
del bus_census['Q2017_4']
del bus_census['Q2019_1']
del bus_census['Q2019_2']


In [ ]:
bus_census

In [ ]:
#narrow datasat to just Phoenix jurisdiction and within the city of phoenix
phx_j = bus_census.loc[:,'Juris'] == 'Phoenix'

phx_juris = bus_census[phx_j]

phx_city = phx_juris.loc[:,'ComputedCity'] == 'Phoenix'
phx_bus = phx_juris[phx_city]

In [ ]:
#narrow dataset to just weekday averages.
#bus_weekday = phx_bus.loc[:,'Day'] == 'Weekday'

#weekday_riders = phx_bus[bus_weekday]

#weekday_riders

In [ ]:
#narrow dataset to just weekend ridership
#bus_weekend = phx_bus.loc[:,'Day'] == 'Weekend'

#weekend_riders = phx_bus[bus_weekend]

#weekend_riders

In [ ]:
#find average ridership for 2018 from the 4 quarters.
phx2018columns = ['Q2018_12', 'Q22018_2', 'Q2018_3', 'Q42018']

phx2018 = phx_bus.reindex(columns = phx2018columns)


In [ ]:
#take the mean of 2018 instead of using 4 quarters and create new column with this data.
df = phx2018.mean(axis=1)
df
phx_bus['2018'] = df
del phx_bus['Q2018_12']
del phx_bus['Q2018_2']
del phx_bus['Q2018_3']
del phx_bus['Q42018']

phx_bus


In [ ]:
#save new csv
phx_bus.rename(columns ={'2018': 'Mean Daily Riders 2018'})
phx_bus.to_csv(r'../PW_CSVs/phx_bus.csv')

In [ ]:
#filter for just weekday ridership
bus_weekday = phx_bus.loc[:,'Day'] == 'Weekday'

weekday_riders = phx_bus[bus_weekday]

weekday_riders

#filter for just weekend ridership
bus_weekend = phx_bus.loc[:,'Day'] == 'Weekend'

weekend_riders = phx_bus[bus_weekend]

del weekend_riders ['Poverty Count']
weekend_riders

In [ ]:
#group data by zipcode and provide agg to analyze by zipcode
weekday_bus = weekday_riders.drop_duplicates( subset='StopID', keep="first", inplace=False)

weekday_bus

weekday = weekday_bus.groupby('Zipcode').agg({
    'StopID':'count',
    'ComputedStreetAddress':'count',
    'ComputedCity':'first',
    'Juris':'first',
    'Day':'first',
    'Per Capita Income': 'mean',
    'Population': 'first',
    'Poverty Rate': 'first',
    '2018':'mean'
})



weekday

#weekday poverty rate by ridership numbers
plt.scatter(weekday['2018'],weekday['Poverty Rate'], marker="o", facecolors="violet", edgecolors="black",
            alpha=0.75)

plt.title("Poverty Rate - Avg Daily Ridership (Weekdays)")
plt.xlabel("Daily Riders")
plt.ylabel("Poverty Rate")

# Run linear regression
(slope, intercept, rvalue, pvalue, stderr) = linregress(weekday['2018'], weekday['Poverty Rate'])
regress_values = weekday['2018'] * slope + intercept

# Plot regression line
plt.plot(weekday['2018'],regress_values,"r-")


#finding R^2

np.corrcoef(weekday.loc[:,'Poverty Rate'], regress_values)[0,1]**2
print(f"The r-squared is: {rvalue**2}")
plt.savefig('../PW_CSVs/poverty_riders_weekday')


In [ ]:
#weekday population by ridership numbers
plt.scatter(weekday['2018'],weekday['Population'], marker="o", facecolors="violet", edgecolors="black",
            alpha=0.75)

plt.title("Population - Avg Daily Ridership (Weekdays)")
plt.xlabel("Daily Riders")
plt.ylabel("Population")

# Run linear regression
(slope, intercept, rvalue, pvalue, stderr) = linregress(weekday['2018'], weekday['Population'])
regress_values = weekday['2018'] * slope + intercept

# Plot regression line
plt.plot(weekday['2018'],regress_values,"r-")


#finding R^2
np.corrcoef(weekday.loc[:,'Population'], regress_values)[0,1]**2
#print R^2
print(f"The r-squared is: {rvalue**2}")

#save figure
plt.savefig('../PW_CSVs/population_riders_weekday')

In [ ]:
#plot per capita income by 2018 rider numbers
plt.scatter(weekday['2018'],weekday['Per Capita Income'], marker="o", facecolors="violet", edgecolors="black",
            alpha=0.75)

plt.title("Per Capita Income - Avg Daily Ridership (Weekdays)")
plt.xlabel("Daily Riders")
plt.ylabel("Per Capita Income")

# Run linear regression
(slope, intercept, rvalue, pvalue, stderr) = linregress(weekday['2018'], weekday['Per Capita Income'])
regress_values = weekday['2018'] * slope + intercept

# Plot regression line
plt.plot(weekday['2018'],regress_values,"r-")

#finding R^2
#print R^2
#save figure
np.corrcoef(weekday.loc[:,'Per Capita Income'], regress_values)[0,1]**2
print(f"The r-squared is: {rvalue**2}")
plt.savefig('../PW_CSVs/income_riders_weekday')

In [ ]:
#group weekend rider numbers by zip code and agg columns values to analyze
weekend_bus = weekend_riders.drop_duplicates( subset='StopID', keep="first", inplace=False)

weekend = weekend_bus.groupby('Zipcode').agg({
    'StopID':'count',
    'ComputedStreetAddress':'count',
    'ComputedCity':'first',
    'Juris':'first',
    'Day':'first',
    'Per Capita Income': 'mean',
    'Population': 'first',
    'Poverty Rate': 'first',
    '2018':'mean'
})

#plot weekend poverty rate by rider #s
plt.scatter(weekend['2018'],weekend['Poverty Rate'], marker="o", facecolors="violet", edgecolors="black",
            alpha=0.75)

plt.title("Poverty Rate - Avg Daily Ridership (Weekend)")
plt.xlabel("Daily Riders")
plt.ylabel("Poverty Rate")

# Run linear regression
(slope, intercept, rvalue, pvalue, stderr) = linregress(weekend['2018'], weekend['Poverty Rate'])
regress_values = weekend['2018'] * slope + intercept

# Plot regression line
plt.plot(weekend['2018'],regress_values,"r-")


#finding R^2
#print R^2
#save figure
np.corrcoef(weekend.loc[:,'Poverty Rate'], regress_values)[0,1]**2

print(f"The r-squared is: {rvalue**2}")
plt.savefig('../PW_CSVs/poverty_riders_weekend')

In [ ]:
#weekend population by rider #s
plt.scatter(weekend['2018'],weekend['Population'], marker="o", facecolors="violet", edgecolors="black",
            alpha=0.75)

plt.title("Population - Avg Daily Ridership (Weekend)")
plt.xlabel("Daily Riders")
plt.ylabel("Population")

# Run linear regression
(slope, intercept, rvalue, pvalue, stderr) = linregress(weekend['2018'], weekend['Population'])
regress_values = weekend['2018'] * slope + intercept

# Plot regression line
plt.plot(weekend['2018'],regress_values,"r-")

line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
plt.annotate(line_eq,(6,10),fontsize=15,color="red")


#finding R^2
#print R^2
#save figure
np.corrcoef(weekend.loc[:,'Population'], regress_values)[0,1]**2

print(f"The r-squared is: {rvalue**2}")

plt.savefig('../PW_CSVs/population_riders_weekend')

In [ ]:
#weekend per capita income by rider numbers
plt.scatter(weekend['2018'],weekend['Per Capita Income'], marker="o", facecolors="violet", edgecolors="black",
            alpha=0.75)

plt.title("Per Capita Income - Avg Daily Ridership (Weekend)")
plt.xlabel("Daily Riders")
plt.ylabel("Per Capita Income")

# Run linear regression
(slope, intercept, rvalue, pvalue, stderr) = linregress(weekend['2018'], weekend['Per Capita Income'])
regress_values = weekend['2018'] * slope + intercept

# Plot regression line
plt.plot(weekend['2018'],regress_values,"r-")

line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
plt.annotate(line_eq,(6,10),fontsize=15,color="red")


#finding R^2
#print R^2
#save figure
np.corrcoef(weekend.loc[:,'Per Capita Income'], regress_values)[0,1]**2

print(f"The r-squared is: {rvalue**2}")

plt.savefig('../PW_CSVs/income_riders_weekend')